# Experiment Proper

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from pprint import pprint
import time

from ctmmodels.const import *

# Add more CTM models here as needed
from ctmmodels.ringbarrier import DTSimplexRingBarrier as NewModel
from ctmmodels.parentmodel import ParentModel as OldModel

In [2]:
IMAGE_PATH = 'graphs/'
DF_PATH = 'dataframes/'

In [3]:
time_range = 30
time_ticks = np.arange(0, time_range+1, 1)

parameters = {
    'r_left': 0.25,
    'r_through': 0.5,
    'r_right': 0.25,
    'sat_flow_rate': 1800,
    'time_range': time_range,
    'time_step': 2,
    'g_min': 6,
    'g_max': 20,
    'flow_rate_reduction': 1
}

In [4]:
def run_model(demand, weights=(0.4, 0.4, 0.2), new_model=True, log_output=True):
    if new_model:
        model = NewModel(
            demand=demand,
            alpha=weights[0],
            beta=weights[1],
            gamma=weights[2],
            **parameters
        )
    else:
        model = OldModel(
            demand=demand,
            **parameters
        )

    model.generate()
    runtime = model.solve(log_output=log_output)
    dfx, dfy, dfg = model.return_solution()
    dfparams = model.return_parameters()
    obj_values = model.return_objective_value()
    return dfx, dfy, dfg, dfparams, obj_values, runtime, model

In [5]:
def save_df(df, filename):
    df.to_pickle(DF_PATH + filename + ".pkl")

## Steps for Experiment Running

1. Generate the initial greentimes and objective values for perusal later in the discussion
2. Use the initial greentimes in running the LocalSim simulator (USE THE SAME PARAMETERS)
3. Record the animation
4. Obtain the results and preprocess for later

## Current Parameters

* **6-20 seconds** greentime range
* **30 time steps** or 60 seconds
* **5 runs** in LocalSim

## 1. Generate initial greentimes

In [6]:
simplex_points = [
    (0.8, 0.1, 0.1),
    (0.1, 0.8, 0.1),
    (0.1, 0.1, 0.8),
    (0.45, 0.45, 0.1),
    (0.1, 0.45, 0.45),
    (0.45, 0.1, 0.45),
    (0.33, 0.33, 0.33),
]

demands = [
    450,
    900,
    (450, 900),
    (900, 1800)
]

In [7]:
# Generate the new model greentimes

_df_tuples = []

for d in demands:
    for pt in simplex_points:
        if d == 450:
            break
        dfx, dfy, dfg, dfparams, objv, runtime, model = run_model(d, weights=pt, new_model=True, log_output=True)
        
        # Save the dataframes
        save_df(dfx, 'experiments/volumes/volumes_d{}_new_a{}_b{}_c{}'.format(d, pt[0], pt[1], pt[2]))
        save_df(dfy, 'experiments/flows/flows_d{}_new_a{}_b{}_c{}'.format(d, pt[0], pt[1], pt[2]))
        save_df(dfg, 'experiments/greentimes/greentimes_d{}_new_a{}_b{}_c{}'.format(d, pt[0], pt[1], pt[2]))
        
        # Save the other results
        if isinstance(d, int):
            d_tuple = (d,d)
        else:
            d_tuple = d

        _df_tuples.append(
            (d_tuple[0], d_tuple[1], runtime, objv[0], objv[1], objv[2], pt[0], pt[1], pt[2])
        )
    
    # Old model
    dfx, dfy, dfg, dfparams, objv, runtime, model = run_model(d, new_model=False, log_output=True)
        
    # Save the dataframes
    save_df(dfx, 'experiments/volumes/volumes_d{}_old'.format(d))
    save_df(dfy, 'experiments/flows/flows_d{}_old'.format(d))
    save_df(dfg, 'experiments/greentimes/greentimes_d{}_old'.format(d))

    # Save the other results
    if isinstance(d, int):
        d_tuple = (d,d)
    else:
        d_tuple = d

    _df_tuples.append(
        (d_tuple[0], d_tuple[1], runtime, objv[0], objv[1], objv[2], 0, 0, 0)
    )

Solving...
CPXPARAM_Read_DataCheck                          1
Tried aggregator 3 times.
MIP Presolve eliminated 4220 rows and 580 columns.
MIP Presolve modified 568 coefficients.
Aggregator did 782 substitutions.
Reduced MIP has 2348 rows, 1458 columns, and 7276 nonzeros.
Reduced MIP has 30 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (17.71 ticks)
Probing time = 0.00 sec. (0.14 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 1128 rows and 714 columns.
Reduced MIP has 1220 rows, 744 columns, and 3974 nonzeros.
Reduced MIP has 30 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (5.25 ticks)
Probing time = 0.00 sec. (0.09 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.03 sec. (21.26 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer 

      0     0     -351.1115   184     -309.5831      Cuts: 34      927   13.41%
      0     2     -351.1115   184     -309.5831     -351.0617      927   13.40%
Elapsed time = 1.63 sec. (1077.17 ticks, tree = 0.01 MB, solutions = 4)
    102    87     -314.9529    64     -309.5831     -349.0109    10848   12.74%
*   312+  221                         -310.3784     -344.6231            11.03%
    326   239     -314.3781    36     -310.3784     -344.6231    23045   11.03%
    577   374        cutoff           -310.3784     -344.3452    32934   10.94%
    833   579     -330.1019   120     -310.3784     -339.8079    44135    9.48%
    954   646     -316.4489    99     -310.3784     -339.7704    47652    9.47%
*   962+  525                         -311.6992     -339.7704             9.01%
*   962+  590                         -311.6992     -339.7704             9.01%
    962   527     -315.7013    61     -311.6992     -339.7704    41071    9.01%
   1160   723     -320.5086    81     -311.6992 

*     0+    0                        -2489.5739    -2531.1851             1.67%
      0     0    -2529.4724   162    -2489.5739     Cuts: 181      535    1.60%
      0     0    -2529.2232   156    -2489.5739      Cuts: 79      625    1.59%
      0     0    -2528.8092   170    -2489.5739      Cuts: 57      717    1.58%
      0     0    -2528.7945   168    -2489.5739      Cuts: 19      748    1.58%
      0     0    -2528.7934   162    -2489.5739    MIRcuts: 2      751    1.58%
      0     0    -2528.7934   162    -2489.5739       Cuts: 6      753    1.58%
*     0+    0                        -2491.9923    -2528.7934             1.48%
      0     2    -2528.7934   162    -2491.9923    -2528.4278      753    1.46%
Elapsed time = 1.57 sec. (1066.45 ticks, tree = 0.01 MB, solutions = 5)
     64    57    -2510.3513   130    -2491.9923    -2525.4613    11340    1.34%
    215   151    -2513.4263   132    -2491.9923    -2522.5541    25781    1.23%
    395   269    -2504.6122    90    -2491.9923 

Probing time = 0.00 sec. (0.55 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 604 rows and 104 columns.
Aggregator did 88 substitutions.
Reduced MIP has 2796 rows, 1714 columns, and 9972 nonzeros.
Reduced MIP has 270 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (5.20 ticks)
Probing time = 0.00 sec. (1.18 ticks)
Tried aggregator 1 time.
Reduced MIP has 2796 rows, 1714 columns, and 9972 nonzeros.
Reduced MIP has 270 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (6.74 ticks)
Probing time = 0.00 sec. (1.18 ticks)
Clique table members: 540.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.16 sec. (170.08 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                         1531.0503    -190

   6779  2182        cutoff          -1404.2481    -1425.2742   358963    1.50%
   7758  2863    -1408.3885    80    -1404.2481    -1424.5906   434057    1.45%
*  7853+ 2886                        -1404.9805    -1424.5405             1.39%
   8929  3405    -1416.7607    89    -1404.9805    -1423.4827   501361    1.32%
   9972  3977    -1405.8979    79    -1404.9805    -1422.5038   566537    1.25%
  11241  4556    -1406.2671    69    -1404.9805    -1421.7806   632233    1.20%
  12561  5099    -1408.4536   103    -1404.9805    -1421.0623   692250    1.14%
Elapsed time = 29.24 sec. (14946.37 ticks, tree = 5.30 MB, solutions = 9)
  13735  5756    -1413.3796   101    -1404.9805    -1420.4468   769760    1.10%
  15033  6261    -1414.0038    94    -1404.9805    -1419.7901   834990    1.05%
* 15481+ 6436                        -1406.2609    -1419.5792             0.95%
  16343  6338    -1414.6802   100    -1406.2609    -1419.2334   905014    0.92%
  17833  6945    -1410.7527   128    -1406.260

...

   1787  1038     -671.0322   143     -633.0450     -679.3387   112402    7.31%
   3013  1702        cutoff           -633.0450     -671.2530   176568    6.04%
Elapsed time = 6.60 sec. (4102.49 ticks, tree = 1.84 MB, solutions = 16)
   3333  2016     -711.7671   218     -633.0450     -669.4835   207425    5.76%
   3350   276     -667.4851   168     -633.0450     -669.4835   210037    5.76%
   3945   386     -658.8036   121     -633.0450     -669.4835   255079    5.76%
   4642   783     -636.1647   110     -633.0450     -669.4835   294817    5.76%
   5538  1399     -660.0830   146     -633.0450     -669.4835   356426    5.76%
   6564  1971     -642.1892   100     -633.0450     -669.4835   412051    5.76%
   7668  2624     -655.2314   107     -633.0450     -667.8604   472814    5.50%
*  7735  2638      integral     0     -635.9419     -667.7711   473256    5.01%
   8972  3032     -662.0340   175     -635.9419     -665.1468   538838    4.59%
  10125  3561     -652.3953   127     -635.9419

   9140  3070     -528.4953   119     -524.5034     -551.1455   400375    5.08%
  10356  3595     -536.1411   121     -524.5034     -548.8349   443972    4.64%
  11538  4013     -534.3303   127     -524.5034     -547.0073   480268    4.29%
Elapsed time = 29.41 sec. (15898.98 ticks, tree = 51.19 MB, solutions = 21)
  12850  4443     -535.7922   125     -524.5034     -545.2241   518642    3.95%
  14038  4848     -533.4294   119     -524.5034     -543.8367   561793    3.69%
  15382  5238     -532.1452   135     -524.5034     -542.5858   606057    3.45%
  16631  5542     -536.0647   107     -524.5034     -541.4373   641051    3.23%
  17873  5824     -530.9104   100     -524.5034     -540.3946   677746    3.03%
  19212  6197     -526.3792    65     -524.5034     -539.5551   724398    2.87%
  20609  6456        cutoff           -524.5034     -538.6389   762714    2.70%
  21903  6681     -529.1237   116     -524.5034     -537.8271   805958    2.54%
  23298  6853     -525.0433    95     -524.5

Total (root+branch&cut) =    2.20 sec. (1500.18 ticks)
Done!
Time elapsed: 2.25904512405
Solving...
CPXPARAM_Read_DataCheck                          1
Tried aggregator 2 times.
MIP Presolve eliminated 3228 rows and 352 columns.
MIP Presolve modified 692 coefficients.
Aggregator did 132 substitutions.
Reduced MIP has 3442 rows, 1856 columns, and 11308 nonzeros.
Reduced MIP has 270 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (12.99 ticks)
Found incumbent of value 504.809674 after 0.03 sec. (22.99 ticks)
Probing fixed 0 vars, tightened 104 bounds.
Probing time = 0.00 sec. (0.55 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 604 rows and 104 columns.
Aggregator did 88 substitutions.
Reduced MIP has 2750 rows, 1664 columns, and 9808 nonzeros.
Reduced MIP has 270 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (5.15 ticks)
Probing time = 0.00 sec. (1.18 ticks)
Tried aggregator 1 time.
Reduced MIP has 2750 rows, 1664 columns, a

Reduced MIP has 270 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.03 sec. (12.99 ticks)
Found incumbent of value 2271.643534 after 0.06 sec. (22.99 ticks)
Probing fixed 0 vars, tightened 104 bounds.
Probing time = 0.00 sec. (0.55 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 604 rows and 104 columns.
Aggregator did 88 substitutions.
Reduced MIP has 2750 rows, 1664 columns, and 9808 nonzeros.
Reduced MIP has 270 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (5.15 ticks)
Probing time = 0.00 sec. (1.18 ticks)
Tried aggregator 1 time.
Reduced MIP has 2750 rows, 1664 columns, and 9808 nonzeros.
Reduced MIP has 270 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (6.60 ticks)
Probing time = 0.00 sec. (1.18 ticks)
Clique table members: 540.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.

MIP Presolve eliminated 604 rows and 104 columns.
Aggregator did 88 substitutions.
Reduced MIP has 2750 rows, 1664 columns, and 9808 nonzeros.
Reduced MIP has 270 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (5.15 ticks)
Probing time = 0.00 sec. (1.18 ticks)
Tried aggregator 1 time.
Reduced MIP has 2750 rows, 1664 columns, and 9808 nonzeros.
Reduced MIP has 270 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (6.61 ticks)
Probing time = 0.00 sec. (1.18 ticks)
Clique table members: 540.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.17 sec. (166.97 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                         2271.6435    -4431.4409           295.08%
      0     0     -171.1935   136     2

In [9]:
df = pd.DataFrame(data=_df_tuples,columns=['demand_ns', 'demand_ew', 'runtime', 'delay', 'throughput', 'objective_value', 'alpha', 'beta', 'gamma'])
save_df(df, "experiments/results2")

## 2. Run the experiments

**Notes:**

* USE THE SAME PARAMETERS
* Label the results Excel sheets later